# Buffer Overflow

- one of the top 25 most dangerous software errors according to Common Weakness Enemuration (CWE) for several years as recently as 2019 [https://cwe.mitre.org/top25/archive/2019/2019_cwe_top25.html](https://cwe.mitre.org/top25/archive/2019/2019_cwe_top25.html)
- most Internet worms use buffer overflow vulnerabilities to propagate
- C and C++ assume that programmers are responsible for data integrity
- modern compilers and OSes provide various protections making it harder to exploit traditional buffer overflows
- once the buffer/memory is allocated, there's no built-in mechanism to safeguard the amount of data that can be stored into
    - this is called buffer overrun or buffer overflow
    - user data may overwrite other part of the memory it's not supposed to making the program behave differently or crashing it
    - attackers could take advantage of this flaw to control the program and may pawn the system by executing shellcode or payload of their choice
- buffer overflow can violate all three core principles of **CIA** triad - **Confidentiality, Integrity, and Availability**
- **Confidentiality**: allows attackers to read confidential or data that they're not supposed to have access to
- **Integrity**: allows attackers to modify part of the data/program that they're not supposed to
- **Availability**: allows attackes to crash the system (SEGFAULT) violating availbility of the service

- buffer can be overflown in all memory segments (Global, Stack and Heap)
- Stack overflow can have the most significant consequences!

## Disable ASLR and other compiler protections
- compile.sh script disables ASLR and compiler protections

In [1]:
# if value is 0; ASLR is disabled
! cat /proc/sys/kernel/randomize_va_space

0


In [ ]:
# run this on terminal
# can't interactively type sudo password!
! echo 0 | sudo tee /proc/sys/kernel/randomize_va_space

[sudo] password for kali: 

## Program's memory and stack may shift
- programs and data must be loaded into RAM inorder to execute
- depending on how the program is launched, several factors make the program's location in memory shift
    - even after virtual-address space randomization is disabled
- e.g. environment variables, length of program name, argument length to program, etc.

### see the difference between stack_overflow_arg.cpp and stack_overflow_stdio.cpp
- stack_overflow_stdio.cpp doesn't take/use any argument
    - interactively prompts user to enter some text duing program execution
- stack_overflow_arg.cpp takes data
    - uses argument provided to the program
    - doesn't prompt user
- let's compile and run these programs to see the address of buffer

In [4]:
%%bash
cp ./booksrc/stack_overflow_stdio.cpp .
cp ./booksrc/stack_overflow_arg.cpp .

In [5]:
! cat stack_overflow_stdio.cpp

#include <stdio.h>
#include <string.h>
#include <sys/types.h>
#include <stdlib.h>
#include <unistd.h>
#include <iostream>
using namespace std;

#define BUFSIZE 128

using namespace std;

char * mgets(char *dst) {
    char *ptr = dst;
	char ch; 
    // int ch;
	/* skip leading white spaces */ 
    // ch = getchar()
	while (cin.get(ch) && ch == ' ' or ch == '\t') 
		; 
 
	if ((ch == '\n') or (ch == EOF)) { 
		*ptr = '\0';
		return dst; 
	} 
	else
		*ptr = ch;
 
	/* now read the rest until \n or EOF */ 
	while (ch = getchar()) {
        if (ch == '\n' or ch == EOF) break;
		*(++ptr) = ch; 
	} 
    *(++ptr) = 0;
    return dst;
}

void bad() {
    char buffer[BUFSIZE];
    printf("buffer is at %p\n", buffer);
    printf("Enter some data: ");
    mgets(buffer);
    //copydata(buffer, data);
    cout << "buffer contains: " << buffer << endl;
    printf("size of buffer = %d and length is %d\n", sizeof(buffer), strlen(buffer));
}

int main(int argc, char *argv[]) {
    // Set the gid to the ef

### buffer address in stack_overflow_stdio.cpp
- let's compile the program and run it

In [8]:
! echo kali | sudo -S ./compile.sh stack_overflow_stdio.cpp stack_overflow_stdio.exe

[sudo] password for kali: 

In [10]:
# since stack_overflow_demo.exe is an interactive program,
# you must pipe some data into the program on Jupyter Notebook
# On terminal, you can either pipe or interactively type when prompted
! echo "Hello_World" | ./stack_overflow_stdio.exe
# Note the address of buffer!

buffer is at 0xffffc2c0
Enter some data: buffer contains: Hello_World
size of buffer = 128 and length is 11
Good bye!


In [11]:
! python -c 'print("A"*100)' | ./stack_overflow_stdio.exe
# Note the address of buffer!

buffer is at 0xffffc2c0
Enter some data: buffer contains: AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
size of buffer = 128 and length is 100
Good bye!


In [12]:
! python -c 'print("A"*200)' | ./stack_overflow_stdio.exe
# Note the address of buffer!

buffer is at 0xffffc2c0
Enter some data: buffer contains: AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
size of buffer = 128 and length is 200
/bin/bash: line 1: 175757 Done                    python -c 'print("A"*200)'
     175758 Segmentation fault      | ./stack_overflow_stdio.exe


- Notice the same address for buffer in stack_overflow_stdio.exe no matter how long the user-entered text is
- this makes it easier to exploit the program as the address of the buffer is alwasy at the same location!

## buffer address in stack_overflow_arg.cpp
- compile and run the program and notice the address of buffer at different location

In [6]:
! cat stack_overflow_arg.cpp

#include <cstdio>
#include <cstring>
#include <iostream>

using namespace std;

#define BUFSIZE 128

void copydata(char *dst_buf, char *src_buf) {
    // similar to strcpy in cstdio
    size_t i = 0;
    // copy each character from src_buf to des_buffer
    // stop when NUL character is encountered
    while (*(src_buf+i) != 0) {
        *(dst_buf+i) = *(src_buf+i);
        i++;
    }
    *(dst_buf+i) = 0; // end with NUL character
}

void bad(char *data) {
    char buffer[BUFSIZE];
    printf("buffer is at %p\n", buffer);
    //strcpy(buffer, data);
    copydata(buffer, data);
    cout << "buffer contains: \n" << buffer << endl;
}

int main(int argc, char *argv[]) {
    bad(argv[1]);
    cout << "Good bye!" << endl;
    return 0;
}

In [13]:
! echo kali | sudo -S ./compile.sh stack_overflow_arg.cpp stack_overflow_arg.exe

[sudo] password for kali: 

In [14]:
# run the program with an argument
# Note you CAN'T pipe the argument into the program as in previous example!
! ./stack_overflow_arg.exe "Hello World"
# Note the address of buffer

buffer is at 0xffffc2b0
buffer contains: 
Hello World
Good bye!


In [21]:
# run the progam with the same length argument
! ./stack_overflow_arg.exe "World_Hello"
# Note the address of buffer; no change because same program is run with the same length of argument

buffer is at 0xffffc2b0
buffer contains: 
World_Hello
Good bye!


In [17]:
# run the program with 100 As
! ./stack_overflow_arg.exe $(python -c 'print("A"*100)')
# Note the address of buffer!

buffer is at 0xffffc250
buffer contains: 
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
Good bye!


In [18]:
# run the program with 200 As
! ./stack_overflow_arg.exe $(python -c 'print("A"*200)')
# Note the address of buffer!

buffer is at 0xffffc1f0
buffer contains: 
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA


## length of the program name
- let's give names of different lengths to the same program and notice the address of buffer
- this is true regardless the program takes an argument or not 
    - NOTE the name of the program itself is passed as an argument to the program as an environment variable

In [19]:
# let's compile the program and run the program with name stack_overflow_stdio.exe
! echo kali | sudo -S ./compile.sh stack_overflow_stdio.cpp stack_overflow_stdio.exe

[sudo] password for kali: 

In [20]:
! echo "Hello_World" | ./stack_overflow_stdio.exe

buffer is at 0xffffc2c0
Enter some data: buffer contains: Hello_World
size of buffer = 128 and length is 11
Good bye!


In [23]:
# if the same program is run multiple times, 
! echo "Hello_World" | ./stack_overflow_stdio.exe
# Notice no change in buffer address

buffer is at 0xffffc2c0
Enter some data: buffer contains: Hello_World
size of buffer = 128 and length is 11
Good bye!


In [24]:
# let's compile nd run the program with name stack.exe
! echo kali | sudo -S ./compile.sh stack_overflow_stdio.cpp stack.exe

[sudo] password for kali: 

In [25]:
! echo "Hello_World" | ./stack.exe

buffer is at 0xffffc2f0
Enter some data: buffer contains: Hello_World
size of buffer = 128 and length is 11
Good bye!


In [26]:
# let's compile and run the program with the name stack1.exe
! echo kali | sudo -S ./compile.sh stack_overflow_stdio.cpp stack1.exe
# minor change in length may not shift it

[sudo] password for kali: 

In [27]:
! echo "Hello_World" | ./stack1.exe

buffer is at 0xffffc2f0
Enter some data: buffer contains: Hello_World
size of buffer = 128 and length is 11
Good bye!


In [28]:
# let's compile the program and run the program with the name stack11.exe
! echo kali | sudo -S ./compile.sh stack_overflow_stdio.cpp stack11.exe

[sudo] password for kali: 

In [29]:
! echo "Hello_World" | ./stack11.exe
# perhaps, filename is not long engouh to make a difference

buffer is at 0xffffc2f0
Enter some data: buffer contains: Hello_World
size of buffer = 128 and length is 11
Good bye!


In [31]:
%%bash
# let's compile the program with output program name stack11111.exe
echo kali | sudo -S ./compile.sh stack_overflow_stdio.cpp stack11111.exe
echo "Hello_World" | ./stack11111.exe

buffer is at 0xffffc2e0
Enter some data: buffer contains: Hello_World
size of buffer = 128 and length is 11
Good bye!


[sudo] password for kali: 

In [33]:
# let's find the no. of bytes the buffer address was shifted by
print(0xffffc2f0-0xffffc2e0)

16


## Run program with modified environment
- **env** command let's you run the program in a modified environment
- **env -i** - ignores environmen or starts a program without one

In [38]:
! echo Hello | ./stack_overflow_stdio.exe

buffer is at 0xffffc2c0
Enter some data: buffer contains: Hello
size of buffer = 128 and length is 5
Good bye!


In [40]:
! echo Hello | env -i ./stack_overflow_stdio.exe

buffer is at 0xffffdda0
Enter some data: buffer contains: Hello
size of buffer = 128 and length is 5
Good bye!


In [41]:
# let's look at the current environment
! env

SHELL=/bin/bash
SESSION_MANAGER=local/kali:@/tmp/.ICE-unix/815,unix/kali:/tmp/.ICE-unix/815
WINDOWID=0
QT_ACCESSIBILITY=1
GCC_RANLIB=/home/kali/miniconda3/bin/x86_64-conda_cos6-linux-gnu-gcc-ranlib
XDG_CONFIG_DIRS=/etc/xdg
XDG_SESSION_PATH=/org/freedesktop/DisplayManager/Session0
XDG_MENU_PREFIX=xfce-
CONDA_EXE=/home/kali/miniconda3/bin/conda
_CE_M=
LANGUAGE=
LESS_TERMCAP_se=
GPROF=/home/kali/miniconda3/bin/x86_64-conda_cos6-linux-gnu-gprof
LESS_TERMCAP_so=
_CONDA_PYTHON_SYSCONFIGDATA_NAME=_sysconfigdata_x86_64_conda_cos6_linux_gnu
SSH_AUTH_SOCK=/tmp/ssh-1OINaY2I3Or6/agent.815
STRINGS=/home/kali/miniconda3/bin/x86_64-conda_cos6-linux-gnu-strings
CPP=/home/kali/miniconda3/bin/x86_64-conda_cos6-linux-gnu-cpp
DESKTOP_SESSION=lightdm-xsession
SSH_AGENT_PID=844
XDG_SEAT=seat0
PWD=/home/kali/projects/Hacking-CPP-Notebooks
LOGNAME=kali
XDG_SESSION_DESKTOP=lightdm-xsession
QT_QPA_PLATFORMTHEME=qt5ct
XDG_SESSION_TYPE=x11
CONDA_PREFIX=/home/kali/miniconda3
PANEL_GDK_CORE_DEVICE_EVENTS=0
CXX=/hom

In [43]:
! ./stack_overflow_arg.exe Hello

buffer is at 0xffffc2b0
buffer contains: 
Hello
Good bye!


In [44]:
! env -i ./stack_overflow_arg.exe Hello

buffer is at 0xffffdd80
buffer contains: 
Hello
Good bye!


In [46]:
# argument length and program length still matters
# let's try with longer argument...
! env -i ./stack_overflow_arg.exe "Hello there beautiful world!"

buffer is at 0xffffdd60
buffer contains: 
Hello there beautiful world!
Good bye!
